In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pymorphy3
import re
from tqdm import tqdm
import umap

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier

# Разложение матрицы на три с сокращением размерности.
from sklearn.decomposition import TruncatedSVD

In [20]:
news_texts = pd.DataFrame(pd.read_csv("news.CSV"), columns=['title', 'text'])
news_texts

,title,text
0,"""Калашников"" выпустит партию обновленных АК-12...",\n Во втором полуго...
1,"""Калашников"" начал поставлять снайперскую винт...",\n Поставки новейше...
2,"""Черный май"": как немецкие подводники потерпел...",\n 24 мая исполняет...
3,В РФ продолжают испытывать новый ракетный комп...,\n В России продолж...
4,"За что командир советской подлодки ""Щ-408"" пол...",\n 22 мая исполняет...
...,...,...
1770,ШААЗ выпустил первый отечественный 14-тонный п...,\n «Шадринский авто...
1771,Деятельность «Ростерминалугля» признана эколог...,\n «Ростерминалугол...
1772,«Перспектива» для заслуженного отдыха,\n Почему у нашего ...
1773,«Кузбассразрезуголь» создал первый профстандар...,\n «Кузбассразрезуг...


In [21]:
targets = pd.Series(pd.read_csv('news.CSV')['theme'])
targets

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1770    3.0
1771    3.0
1772    3.0
1773    3.0
1774    3.0
Name: theme, Length: 1775, dtype: float64

In [22]:
%%time
# Считаем матрицу термин-документ, но не с частотами, а со значениями Tf*Idf
X = TfidfVectorizer().fit_transform(news_texts['text'])
# Проводим SVD-разложение по 20 компонентам.
svd = TruncatedSVD(n_components=20)
X2 = svd.fit_transform(X)

explained_variance = svd.explained_variance_ratio_.sum()
print(f"Explained variance of the SVD step: {int(explained_variance * 100)}%")
# Просто чтобы посмотреть, что там в самом деле вектора.
print(X2.shape, X2)

Explained variance of the SVD step: 8%
(1775, 20) [[ 1.18273873e-01 -1.48476552e-02  4.42085260e-02 ... -1.42706390e-02
  -1.59824116e-02  6.78053817e-05]
 [ 1.38709574e-01 -2.92026518e-02  6.95658865e-02 ... -5.50139314e-03
  -3.95390821e-02 -2.47727999e-02]
 [ 2.58704440e-01 -1.16852813e-01 -3.77281416e-02 ...  8.72860868e-02
  -2.36619761e-02 -4.62952337e-02]
 ...
 [ 2.02878586e-01  1.53843086e-02 -2.82683305e-02 ...  1.79963909e-02
   1.65585093e-03 -3.05940286e-02]
 [ 2.91129457e-01  3.55962539e-01  2.03465035e-02 ... -1.40852344e-02
   4.54465014e-02 -1.00629148e-01]
 [ 2.88926067e-01  3.41501725e-01  4.57943518e-03 ...  1.14079449e-02
  -9.31827430e-03  4.84615604e-03]]
CPU times: total: 1.22 s
Wall time: 1.11 s


In [23]:
X

<1775x76252 sparse matrix of type '<class 'numpy.float64'>'
	with 487020 stored elements in Compressed Sparse Row format>

In [24]:
# Список значимых частей речи. 
# Они нам потом понадобятся в немного другом виде. Так что сделаем словарь. чтобы два раза не вставать.
conv_pos = {'ADJF':'ADJ', 'ADJS':'ADJ', 'ADV':'ADV', 'NOUN':'NOUN', 
            'VERB':'VERB', 'PRTF':'ADJ', 'PRTS':'ADJ', 'GRND':'VERB'}

tmp_dict = {} # Кеш значимых слов.
nones = {} # Кеш незначимых слов.

morph = pymorphy3.MorphAnalyzer()

# Фильтруем по части речи и возвращаем только начальную форму.
def normalizePymorphy(text, need_pos=True):
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    words = []
    for t in tokens:
        # Если токен уже был закеширован, быстро возьмем результат из него.
        if t in tmp_dict.keys():
            words.append(tmp_dict[t])
        # Аналогично, если он в кеше незначимых слов.
        elif t in nones.keys():
            pass
        # Слово еще не встретилось, будем проводить медленный морфологический анализ.
        else:
            pv = morph.parse(t)
            if pv[0].tag.POS != None:
                if pv[0].tag.POS in conv_pos.keys():
                    if need_pos:
                        word = pv[0].normal_form+"_"+conv_pos[pv[0].tag.POS]
                    else:
                        word = pv[0].normal_form
                    # Отправляем слово в результат, ...
                    words.append(word)
                    # ... и кешируем результат его разбора.
                    tmp_dict[t] = word
                else:
                    # Для незначимых слов можно даже ничего не хранить. Лишь бы потом не обращаться к морфологии.
                    nones[t] = ""
                    
    return words

In [25]:
%%time
# Приведем слова в текстах к начальным формам.
news_texts['NText'] = news_texts['text'].map(lambda x:' '.join(normalizePymorphy(x)))

CPU times: total: 7.09 s
Wall time: 7.11 s


In [26]:

%%time
X_l = TfidfVectorizer().fit_transform(news_texts['NText'])
svd = TruncatedSVD(n_components=10)
X2_l = svd.fit_transform(X)

CPU times: total: 672 ms
Wall time: 629 ms


In [28]:
targets = np.array(targets)
targets

array([0., 0., 0., ..., 3., 3., 3.])

In [29]:
def classify_texts(data, target):
    # Делим данные на обучающую и проверочную выборки.
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=333)
    #Обучаем классификатор и оцениваем точность результатов.
    tree = RandomForestClassifier(criterion='entropy', random_state=333)
    tree.fit(X_train, y_train)
    y_hat = tree.predict(X_test)
    print(f"accuracy = {accuracy_score(y_hat, y_test)}")
    print(confusion_matrix(y_test, y_hat))

In [30]:
classify_texts(X, targets)

accuracy = 0.9464788732394366
[[ 95   9   0   0]
 [  0 100   1   1]
 [  3   1  48   2]
 [  0   2   0  93]]


In [31]:

%%time
# TF*Idf с лемматизацией.
classify_texts(X_l, targets)

accuracy = 0.952112676056338
[[104   0   0   0]
 [  1  99   1   1]
 [  4   6  42   2]
 [  0   2   0  93]]


In [32]:
%%time
# SVD без лемматизации.
classify_texts(X2, targets)

accuracy = 0.971830985915493
[[103   1   0   0]
 [  2  98   1   1]
 [  2   2  50   0]
 [  0   1   0  94]]
CPU times: total: 484 ms
Wall time: 482 ms


In [33]:
%%time
# SVD с лемматизацией.
classify_texts(X2_l, targets)

accuracy = 0.9690140845070423
[[103   1   0   0]
 [  1  99   1   1]
 [  4   2  48   0]
 [  0   1   0  94]]
CPU times: total: 344 ms
Wall time: 370 ms
